# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. Имеются исторические данные: технические характеристики, комплектации и цены автомобилей. Нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Признаки**

- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

**Целевой признак**

- Price — цена (евро)

**План работы**

- Загрузить данные и провести предобработку.
- Выполнить сравнение моделей с использованием различных наборов гиперпараметров.
- Выбрать лучшую модель по результатам метрики RMSE и времени обучения.
- Оформить проект и написать выводы.

## Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from catboost import Pool, CatBoostRegressor
from lightgbm import LGBMRegressor
import time
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("/datasets/autos.csv")
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [3]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

In [4]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [5]:
data.describe(include=['object', 'bool'])

,DateCrawled,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired,DateCreated,LastSeen
count,354369,316879,334536,334664,321474,354369,283215,354369,354369
unique,271174,8,2,250,7,40,2,109,179150
top,2016-03-24 14:49:47,sedan,manual,golf,petrol,volkswagen,no,2016-04-03 00:00:00,2016-04-06 13:45:54
freq,7,91457,268251,29232,216352,77013,247161,13719,17


In [6]:
data.shape

(354369, 16)

In [7]:
def unique_values(df):
    for column in data.columns:
        print('Уникальные значения столбца', column)
        print(df[column].unique())
unique_values(data)

Уникальные значения столбца DateCrawled
['2016-03-24 11:52:17' '2016-03-24 10:58:45' '2016-03-14 12:52:21' ...
 '2016-03-21 09:50:58' '2016-03-14 17:48:27' '2016-03-19 18:57:12']
Уникальные значения столбца Price
[  480 18300  9800 ... 12395 18429 10985]
Уникальные значения столбца VehicleType
[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']
Уникальные значения столбца RegistrationYear
[1993 2011 2004 2001 2008 1995 1980 2014 1998 2005 1910 2016 2007 2009
 2002 2018 1997 1990 2017 1981 2003 1994 1991 1984 2006 1999 2012 2010
 2000 1992 2013 1996 1985 1989 2015 1982 1976 1983 1973 1111 1969 1971
 1987 1986 1988 1970 1965 1945 1925 1974 1979 1955 1978 1972 1968 1977
 1961 1960 1966 1975 1963 1964 5000 1954 1958 1967 1959 9999 1956 3200
 1000 1941 8888 1500 2200 4100 1962 1929 1957 1940 3000 2066 1949 2019
 1937 1951 1800 1953 1234 8000 5300 9000 2900 6000 5900 5911 1933 1400
 1950 4000 1948 1952 1200 8500 1932 1255 3700 3800 4800 1942 7000 1935
 1936 6500 1923 2290

In [8]:
data.duplicated().sum()

4

### Вывод: 
**Первичные наблюдения:**

- В данных есть пропуски 
- Есть небольшое количество дубликатов
- Средняя стоимость авто 2700 евро
- Минимальный год регистриции автомобиля 1000, максимальный 9999. Нужно обозначить диапазон из процентилей.
- В столбце Power присутвует некорректное значение мощности. Максимальное значение в мощности автомобиля 20 000. Нужно обозначить диапазон из процентилей.
- В столбце RegistrationYear существуют некорректные значения года регистрации.
- В столбце RegistrationMonth отсутсвует месяц регистрации.
- Минимальный пробег авто 5000, максимальный 150000 причем много авто имееют максимальный пробег.
- 8 уникальных типов автомобильного кузова
- Чаще встречаются авто с ручной коробкой передач
- Чаще встречаются автомобиле на бензине
- Много пропусков в столбце о ремонте авто, но чаще встречаютс автомобили без ремонта

**Значимые признаки**

- VehicleType - Тип автомобильного кузова определяет ее функционал. Столбец влияет на цену.
- Gearbox -Тип коробки передач, если у авто автоматическая коробка - его стоимость выше. 
- Power - Мощность влияет на цену.
- Kilometer - Пробег имеет обратно пропорциональную зависиммость от цены. Больше пробег - меньше цена.
- FuelType - Тип топлива, думаю, тоже влияет на цену.
- Brand и Model (сюда же можно отнести и VehicleType) -  Может влиять на популярность у покупателей, поэтому оставляем. 
- NotRepaired - Была машина в ремонте или нет однозначно влияет на цену.
- RegistrationYear - Год регистрации авто имеет обратно пропорциональную зависиммость от цены. Больше возраст авто - меньше цена.

Остальные столбцы для модели не требуются, так как не влияют на целевой признак.

## Обработка и анализ данных

In [9]:
#Удалим ненужные столбцы
data = data.drop(['DateCrawled', 'LastSeen', 'DateCreated', 'NumberOfPictures', 'PostalCode','RegistrationMonth'],axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       316879 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           334536 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             334664 non-null  object
 6   Kilometer         354369 non-null  int64 
 7   FuelType          321474 non-null  object
 8   Brand             354369 non-null  object
 9   NotRepaired       283215 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


In [10]:
data.isna().sum().sort_values(ascending=False)

NotRepaired         71154
VehicleType         37490
FuelType            32895
Gearbox             19833
Model               19705
Price                   0
RegistrationYear        0
Power                   0
Kilometer               0
Brand                   0
dtype: int64

In [11]:
#Решила заполнить "не указано", так как много пропущеных значений и заполнять "да" или "нет" считаю нецелесообразным
data['NotRepaired'] = data['NotRepaired'].fillna('not_specified')

**Остальные столбцы имеют пропуски возможно по причине проблем с выгрузкой данных, так как на эти вопросы может ответить только владельцы автомобилей. Заполним пропуски модами**

In [12]:
data["VehicleType"] = data["VehicleType"].fillna(data["VehicleType"].mode().values[0])
data["Gearbox"] = data["Gearbox"].fillna(data["Gearbox"].mode().values[0])
data["Model"] = data["Model"].fillna(data["Model"].mode().values[0])
data["FuelType"] = data["FuelType"].fillna(data["FuelType"].mode().values[0])

In [13]:
#Проверим, что значения категорийных признаков не дублируют друг друга
objects_columns = ["VehicleType","Gearbox","Model","FuelType","Brand","NotRepaired"]
for column in objects_columns:
    print(column, pd.Series(data[column].unique()).duplicated().sum())

VehicleType 0
Gearbox 0
Model 0
FuelType 0
Brand 0
NotRepaired 0


In [14]:
#Так как у нас есть дубликаты, удалим их
data = data.drop_duplicates()
data.duplicated().sum()

0

In [15]:
data.describe()

,Price,RegistrationYear,Power,Kilometer
count,305204.000000,305204.000000,305204.000000,305204.000000
mean,4525.608724,2004.377767,111.619222,127033.639795
std,4574.538221,93.089385,202.148108,38620.754142
min,0.000000,1000.000000,0.000000,5000.000000
25%,1100.000000,1999.000000,69.000000,125000.000000
50%,2800.000000,2003.000000,105.000000,150000.000000
75%,6500.000000,2008.000000,143.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


**Проанализируем значения числовых признаков перед очисткой выбросов**

- Price обладает высоким стандартным отклонениям, что плохо будет влиять на работу алгоритма, также признак нуждается в поиске выбросов
- RegistrationYear - присутствуют аномальные значения, для того чтобы избавиться от аномалий установим диапазон, в котором может находится значение этого столбца - год регистрации не может быть больше 2020, и не может быть меньше 1930. Все показания выше или ниже этих значений приведем к максимальной или минимальной границе, соответственно признак не нуждается в поиске выбросов.
- Power - присутствуют аномальные значения (20 000 лошадиных сил), ограничим их мощностью 3500 лошадиных сил (нашла в интернете самую максимальную мощность) , также нужно искать выбросы.
- Kilometer имеет достаточно реальные значения, поэтому признак не нуждается в поиске выбросов

In [16]:
def RegistrationYear(value):
    if value > 2020:
        return 2020
    elif value < 1930:
        return 1930
    else:
        return value
data["RegistrationYear"] = data["RegistrationYear"].apply(RegistrationYear)

In [17]:
data.loc[data['Power'] > 3500, 'Power'] = 3500

In [18]:
def shore_clearing(data,column):
    Q25=np.array(data[column].quantile(0.25))
    Q75=np.array(data[column].quantile(0.75))
    stat_1=Q25-1.5*(Q75-Q25)
    stat_2=Q75+1.5*(Q75-Q25)
    index_del = []
    for index_value, value in zip(data[column].index,data[column]):
        if stat_2 <= value or value <= stat_1:
            index_del.append(index_value)
    print('Количество строк, выбранных для удаления ' + str(column)+":",len(index_del))
    return index_del

In [19]:
array_num_columns = ["Price","Power"]
count = 0 
for column in array_num_columns:
    del_index = shore_clearing(data,column)
    count += len(del_index)
    data = data.drop(del_index,axis = 0)
print("Было удалено:", count)

Количество строк, выбранных для удаления Price: 16209
Количество строк, выбранных для удаления Power: 5430
Было удалено: 21639


In [20]:
data.corr()

,Price,RegistrationYear,Power,Kilometer
Price,1.000000,0.354863,0.417300,-0.275082
RegistrationYear,0.354863,1.000000,0.088263,-0.124761
Power,0.417300,0.088263,1.000000,0.148650
Kilometer,-0.275082,-0.124761,0.148650,1.000000


### Вывод

- Были найдены и удалены дубликаты
- Пропуски в категориальных признаках были заполнены модой
- Данные были отчищены от выбросов, потеряли довольно много объектов, поэтому, в случае низких результатов модели будет целесообразно изменить коэффициент при межквартильном размахе, что уменьшит общий объем объектов - выбросов.
- Проверена корреляция признаков, наиболее коррелирующим признаком является мощность.

## Обучение моделей

**Для обучения моделей:**
- 1. Изменим типы на category для работы с моделями Catboost и lightGBM - cat, кодировка методом OHE -ohe, кодировка методом OE -ord.
- 2. Разделим выборку на обучающую, тестовую и валидационную выборки
- 3. Проверим модели : линейную регрессию, CatBoost, LightGBM

In [21]:
#del features_train

In [22]:
X = data.drop('Price', axis=1)
y = data['Price']

In [23]:
categ_columns = X.select_dtypes(include='object').columns
# Изменение типа на 'category'
X_cat = X.copy()
X_cat[categ_columns] = X_cat[categ_columns].astype('category')
X_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283565 entries, 0 to 354368
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   VehicleType       283565 non-null  category
 1   RegistrationYear  283565 non-null  int64   
 2   Gearbox           283565 non-null  category
 3   Power             283565 non-null  int64   
 4   Model             283565 non-null  category
 5   Kilometer         283565 non-null  int64   
 6   FuelType          283565 non-null  category
 7   Brand             283565 non-null  category
 8   NotRepaired       283565 non-null  category
dtypes: category(6), int64(3)
memory usage: 10.6 MB


In [24]:
# Кодировка методом One Hot Encoding
X_ohe = pd.get_dummies(X, drop_first=True)
X_ohe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283565 entries, 0 to 354368
Columns: 306 entries, RegistrationYear to NotRepaired_yes
dtypes: int64(3), uint8(303)
memory usage: 90.6 MB


In [25]:
# Кодировка методом Ordinal Encoding
enc = OrdinalEncoder()

X_ord = X.copy()
X_ord[categ_columns] = enc.fit_transform(X_ord[categ_columns])
X_ord.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283565 entries, 0 to 354368
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   VehicleType       283565 non-null  float64
 1   RegistrationYear  283565 non-null  int64  
 2   Gearbox           283565 non-null  float64
 3   Power             283565 non-null  int64  
 4   Model             283565 non-null  float64
 5   Kilometer         283565 non-null  int64  
 6   FuelType          283565 non-null  float64
 7   Brand             283565 non-null  float64
 8   NotRepaired       283565 non-null  float64
dtypes: float64(6), int64(3)
memory usage: 21.6 MB


In [26]:
#Разделим данные на обучающую и тестовую выборки, в тестовую выборку включим 10% данных

y_train, y_test = train_test_split(y, test_size=0.1, random_state=12345)
y_train.shape[0] / y_test.shape[0]

8.999823676693586

In [27]:
X_cat_train, X_cat_test = train_test_split(X_cat, test_size=0.1, random_state=12345)
X_ohe_train, X_ohe_test = train_test_split(X_ohe, test_size=0.1, random_state=12345)
X_ord_train, X_ord_test = train_test_split(X_ord, test_size=0.1, random_state=12345)

In [28]:
# набор масштабированных данных

scaler = MinMaxScaler()

scaler.fit(X_ohe_train)
X_ohe_scal_train = pd.DataFrame(scaler.transform(X_ohe_train), columns=X_ohe_train.columns, index=X_ohe_train.index)
X_ohe_scal_test = pd.DataFrame(scaler.transform(X_ohe_test), columns=X_ohe_test.columns, index=X_ohe_test.index)

scaler.fit(X_ord_train)
X_ord_scal_train = pd.DataFrame(scaler.transform(X_ord_train), columns=X_ord_train.columns, index=X_ord_train.index)
X_ord_scal_test = pd.DataFrame(scaler.transform(X_ord_test), columns=X_ord_test.columns, index=X_ord_test.index)

In [29]:
# RMSE константной модели для обучающей выборки
mean_squared_error(y_train, pd.Series(y_train.mean(), index=y_train.index)) ** 0.5

3501.9236734017

In [30]:
# RMSE константной модели для тестовой выборки
mean_squared_error(y_test, pd.Series(y_train.mean(), index=y_test.index)) ** 0.5

3520.1485042584

### Линейная регрессия

In [31]:
%%time
lr_ohe = LinearRegression()
rmse_lr_ohe = (cross_val_score(estimator=lr_ohe, 
                             X=X_ohe_train, 
                             y=y_train, 
                             cv=5,
                             scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('RMSE Модели Линейной Регрессии (кодировка OHE) =', rmse_lr_ohe)

RMSE Модели Линейной Регрессии (кодировка OHE) = 2346.62535573825
CPU times: user 1min 10s, sys: 31.4 s, total: 1min 41s
Wall time: 1min 41s


In [32]:
%%time
lr_ord = LinearRegression()
rmse_lr_ord = (cross_val_score(estimator=lr_ord, 
                             X=X_ord_train, 
                             y=y_train, 
                             cv=5,
                             scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('RMSE Модели Линейной Регрессии (кодировка OrdinalEncoding) =', rmse_lr_ord)

RMSE Модели Линейной Регрессии (кодировка OrdinalEncoding) = 2598.6610989304936
CPU times: user 608 ms, sys: 361 ms, total: 969 ms
Wall time: 946 ms


In [33]:
%%time
lr_ord_scal = LinearRegression()
rmse_lr_ord_scal = (cross_val_score(estimator=lr_ord_scal, 
                             X=X_ord_scal_train, 
                             y=y_train, 
                             cv=5,
                             scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('RMSE Модели Линейной Регрессии (кодировка OrdinalEncoding с масштабированием) =', rmse_lr_ord_scal)

RMSE Модели Линейной Регрессии (кодировка OrdinalEncoding с масштабированием) = 2598.661098930496
CPU times: user 525 ms, sys: 401 ms, total: 926 ms
Wall time: 990 ms


Качество Линейной Модели на данных, обработанных методом OHE по метрике RMSE выше, чем по методу OrdinalEncoding. В то же время обучение потребовало значительно больше времени.Масштабирование не оказывает влияния на качество модели Линейной Регрессии.

### CatBoostRegressor

In [34]:
%%time

cbr = CatBoostRegressor(cat_features=categ_columns.tolist())
hyperparams_cbr = { 'depth'         : [10, 20, 30],
                    'iterations'    : [100, 200, 300],
                    'random_state'  : [12345],
                    'verbose'       : [False]
                  }
cbr_grid = GridSearchCV(estimator=cbr, 
                        param_grid = hyperparams_cbr, 
                        cv = 3, 
                        scoring ='neg_mean_squared_error',
                        n_jobs=-1)
cbr_grid.fit(X_cat_train, y_train) 

CPU times: user 10min 28s, sys: 4.61 s, total: 10min 33s
Wall time: 10min 43s


GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostRegressor object at 0x7ff300c57fd0>,
             n_jobs=-1,
             param_grid={'depth': [10, 20, 30], 'iterations': [100, 200, 300],
                         'random_state': [12345], 'verbose': [False]},
             scoring='neg_mean_squared_error')

In [35]:
cbr_grid.best_params_

{'depth': 10, 'iterations': 300, 'random_state': 12345, 'verbose': False}

In [36]:
rmse_cbr = (cbr_grid.best_score_*-1) ** 0.5
print('RMSE CatBoostRegressor =', rmse_cbr) 

RMSE CatBoostRegressor = 1484.2096281480683


Качество CatBoostRegressor оказалось выше Линейной Регрессии, но время...

### LGBMRegressor

In [37]:
%%time
lgbmr = LGBMRegressor(categorical_features=categ_columns.tolist())
hyperparams_lgbmr = { 'max_depth'     : [10, 20, 30],
                      'n_estimators'  : [100, 200, 300],
                      'random_state'  : [12345]
                  }
lgbmr_grid = GridSearchCV(estimator=lgbmr, 
                        param_grid = hyperparams_lgbmr, 
                        cv = 3, 
                        scoring ='neg_mean_squared_error',
                        n_jobs=-1)
lgbmr_grid.fit(X_cat_train, y_train) 

CPU times: user 4min 40s, sys: 0 ns, total: 4min 40s
Wall time: 4min 42s


GridSearchCV(cv=3,
             estimator=LGBMRegressor(categorical_features=['VehicleType',
                                                           'Gearbox', 'Model',
                                                           'FuelType', 'Brand',
                                                           'NotRepaired']),
             n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30],
                         'n_estimators': [100, 200, 300],
                         'random_state': [12345]},
             scoring='neg_mean_squared_error')

In [38]:
lgbmr_grid.best_params_

{'max_depth': 20, 'n_estimators': 300, 'random_state': 12345}

In [39]:
rmse_lgbmr = (lgbmr_grid.best_score_*-1) ** 0.5
print('RMSE LGBMRegressor =', rmse_lgbmr) 

RMSE LGBMRegressor = 1481.5063711164069


Модели LGBMRegressor и CatBoostRegressor показали схожее значение метрики RMSE на обучающих данных. Модель CatBoost дал несколько лучшее значение метрики, но длительность перебора одинакового количества гиперпараметров заняло у библиотеки LightGBM намного меньше времени.

## Анализ моделей

In [40]:
#изменила функцию (Функция обучает модель на обучающем наборе данных,возвращает метрику качества rmse 
#на тестовом наборе данных, время обучения и предсказания)
def model_result(model, X_train, X_test, y_train=y_train, y_test=y_test):

    start1 = time.time()
    model.fit(X_train, y_train)
    end1 = time.time()
    time_train = end1 - start1

    start2 = time.time()
    y_pred = model.predict(X_test)
    end2 = time.time()
    time_predict = end2 - start2

    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    return round(rmse), round(time_train, 1), round(time_predict, 1)

In [41]:
# Linear (OrdinalEncoding)
lr_ord_result = model_result(lr_ord, X_train=X_ord_train, X_test=X_ord_test)
lr_ord_result

(2591, 0.1, 0.0)

In [42]:
# Linear (OHE)
lr_ohe_result = model_result(lr_ohe, X_train=X_ohe_train, X_test=X_ohe_test)
lr_ohe_result

(2339, 23.8, 0.1)

In [43]:
# CatBoost
cbr_result = model_result(cbr_grid.best_estimator_, X_train=X_cat_train, X_test=X_cat_test)
cbr_result

(1458, 137.2, 0.2)

In [44]:
# LightGBM
lgbmr_result = model_result(lgbmr_grid.best_estimator_, X_train=X_cat_train, X_test=X_cat_test)
lgbmr_result

(1457, 27.5, 0.7)

In [45]:
model_list = [lr_ord_result, lr_ohe_result, cbr_result, lgbmr_result]
rmse_list = [model[0] for model in model_list]
time_train_list = [model[1] for model in model_list]
time_predict_list = [model[2] for model in model_list]
model_name_list = ['Linear (OrdinalEncoding)', 'Linear (OHE)','CatBoost', 'LightGBM' ]
results_df = pd.DataFrame(data={'RMSE':rmse_list, 'Time_train':time_train_list, 'Time_predict':time_predict_list,},
                                                                                index=model_name_list)
results_df

,RMSE,Time_train,Time_predict
Linear (OrdinalEncoding),2591,0.1,0.0
Linear (OHE),2339,23.8,0.1
CatBoost,1458,137.2,0.2
LightGBM,1457,27.5,0.7


### Вывод

- Провели передобработку данных
- Применили для категориальных признаков метод OHE
- Рассмотрели три алгоритма классификации: CatBoostRegressor, LGBMRegressor, Линейная регрессия

Исходя из полученных результатов, можно сделать вывод, что наименьшую ошибку и время обучения выдает LGBMRegressor, хотя ошибка и у CatBoost незнакчительно отличилась. У CatBoost и Линейной регрессии (оба метода) наименьшее время предсказаний.
В условии указано, что решающим факторами при выборе модели являются следующие показатели:

- Время обучения
- Время предсказания
- Качество предсказаний

Я считаю, что обе модели градиентного бустринга хороши по расчету метрики RMSE, также CatBoost имеет меньшее время предсказаний, чем LGBMRegressor, однако выгоднее всего в данном случае смотрится LGBMRegressor, эта модель при одинаковом подборе гиперпараметров обучается быстрее и выдает наименьшую ошибку, соответственно по 2 из 3 решающих факторов рекомендуется именно эта модель.